<a href="https://colab.research.google.com/github/jnardosp/foldwalker/blob/main/hp_model_mcmc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Parte 1:** Convertir una cadena de aminoácidos a un lattice 3D

## Hydrophobic - Polar Lattice Model MonteCarlo sampling

Let's perform an initial MCMC sampling on a HP model to later do approximate counting of proteins.

Aminoacids & H (Hydrophobic) - P (Polar/Neutral) Classification:
| Aminoacid     | Letter | H or P |
|---------------|--------|--------|
| Alanine       | A      | H      |
| Arginine      | R      | P      |
| Asparagine    | N      | P      |
| Aspartate     | D      | P      |
| Cysteine      | C      | H      |
| Glutamine     | Q      | P      |
| Glutamate     | E      | P      |
| Glycine       | G      | P      |
| Histidine     | H      | P      |
| Isoleucine    | I      | H      |
| Leucine       | L      | H      |
| Lysine        | K      | P      |
| Methionine    | M      | H      |
| Phenylalanine | F      | H      |
| Proline       | P      | P      |
| Serine        | S      | P      |
| Threonine     | T      | P      |
| Tryptophan    | W      | H      |
| Tyrosine      | Y      | H      |
| Valine        | V      | H      |

[Aminoacids & HP](https://www.alfa-chemistry.com/resources/hydrophobicity-index-table-of-common-amino-acids.html)


In [7]:
import plotly.graph_objects as go
import numpy as np
from typing import List, Tuple, Optional

In [8]:
# HP classification mapping
HP_MAP = {
    'A': 'H', 'C': 'H', 'I': 'H', 'L': 'H', 'M': 'H',
    'F': 'H', 'W': 'H', 'Y': 'H', 'V': 'H',
    'R': 'P', 'N': 'P', 'D': 'P', 'Q': 'P', 'E': 'P',
    'G': 'P', 'H': 'P', 'K': 'P', 'P': 'P', 'S': 'P', 'T': 'P'
}

def generate_linear_chain(sequence: str) -> Tuple[List[Tuple[int, int, int]], List[str]]:
    """
    Generate a linear chain along the x-axis (simpler than random SAW).
    
    This provides a better starting point for MCMC than a random SAW.
    """
    coords = []
    hp_types = []
    
    for i, aa in enumerate(sequence):
        coords.append((i, 0, 0))
        hp_types.append(HP_MAP.get(aa, 'P'))
    
    return coords, hp_types

def plot_hp_lattice_3d(sequence, coords, hp_types):
    """Plot 3D HP lattice using Plotly."""
    # Separate H and P residues
    h_coords = [coords[i] for i, hp in enumerate(hp_types) if hp == 'H']
    p_coords = [coords[i] for i, hp in enumerate(hp_types) if hp == 'P']

    # Create figure
    fig = go.Figure()

    # Add hydrophobic residues (H) as red spheres
    if h_coords:
        h_x, h_y, h_z = zip(*h_coords)
        fig.add_trace(go.Scatter3d(
            x=h_x, y=h_y, z=h_z,
            mode='markers',
            marker=dict(size=12, color='red'),
            name='Hydrophobic (H)',
            text=[f'Residue {i+1}: {sequence[i]} (H)'
                  for i, hp in enumerate(hp_types) if hp == 'H']
        ))

    # Add polar residues (P) as blue spheres
    if p_coords:
        p_x, p_y, p_z = zip(*p_coords)
        fig.add_trace(go.Scatter3d(
            x=p_x, y=p_y, z=p_z,
            mode='markers',
            marker=dict(size=10, color='blue', opacity=0.8),
            name='Polar (P)',
            text=[f'Residue {i+1}: {sequence[i]} (P)'
                  for i, hp in enumerate(hp_types) if hp == 'P']
        ))

    # Add connections between residues
    x_coords, y_coords, z_coords = zip(*coords)
    fig.add_trace(go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='lines',
        line=dict(color='gray', width=4),
        name='Protein Backbone'
    ))

    # Update layout
    fig.update_layout(
        title=f'3D HP Lattice Model: {sequence}',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube',
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
        ),
        margin=dict(l=0, r=0, b=0, t=40),
        showlegend=True
    )

    fig.show()

    # Print statistics
    print(f"Sequence: {sequence}")
    print(f"Length: {len(sequence)} residues")
    print(f"H residues: {hp_types.count('H')}")
    print(f"P residues: {hp_types.count('P')}")
    print(f"Unique positions: {len(set(coords))}")

# Example usage
if __name__ == "__main__":
    # Test sequence - you can replace with your own
    test_sequence = "ACDEFGHIKLMNPQRSTVWY"

    coords, hp_types = generate_linear_chain(test_sequence)
    print("Coords: ", coords)
    print("HP_types: ", hp_types)

    # Generate and plot
    plot_hp_lattice_3d(test_sequence, coords, hp_types)

Coords:  [(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0), (6, 0, 0), (7, 0, 0), (8, 0, 0), (9, 0, 0), (10, 0, 0), (11, 0, 0), (12, 0, 0), (13, 0, 0), (14, 0, 0), (15, 0, 0), (16, 0, 0), (17, 0, 0), (18, 0, 0), (19, 0, 0)]
HP_types:  ['H', 'H', 'P', 'P', 'H', 'P', 'P', 'H', 'P', 'H', 'H', 'P', 'P', 'P', 'P', 'P', 'P', 'H', 'H', 'H']


Sequence: ACDEFGHIKLMNPQRSTVWY
Length: 20 residues
H residues: 9
P residues: 11
Unique positions: 20


## **Parte 2:** calcular la función de energía de un lattice 3D

In [11]:
import numpy as np
from itertools import combinations
from typing import Sequence, Union, Callable, Tuple, List, Dict, Optional

def _default_U(a: str, b: str) -> float:
    """Interacción por defecto: -1 si ambos H, 0 en otro caso (modelo HP)."""
    return -1.0 if a.upper() == 'H' and b.upper() == 'H' else 0.0

def energy_hp(coords: Union[np.ndarray, Sequence[Sequence[float]]],
              seq: Union[str, Sequence[str]],
              U: Optional[Union[Callable[[str,str], float], Dict[str, Dict[str, float]]]] = None,
              contact_distance: float = 1.0,
              tol: float = 1e-8,
              metric: str = 'euclidean',
              return_contacts: bool = False
              ) -> Union[float, Tuple[float, List[Tuple[int,int,float]]]]:
    """
    Calcula la energía E_HP(C) = sum_{i=0}^{N-3} sum_{j=i+2}^{N-1} U(h_i,h_j) * C_ij,
    donde C_ij = 1 si i y j son vecinos más cercanos en la retícula (distancia == contact_distance),
    y U(h_i,h_j) es la potencial de interacción (por defecto: -1 si ambos 'H', 0 en otro caso).

    Parámetros:
    - coords: array-like (N,3) con coordenadas de cada residuo (enteras o floats).
    - seq: secuencia de tipos (ej. "HHPHP" o ['H','P',...]).
    - U: función U(a,b)->float o dict-of-dicts U[a][b]; si None usa el modelo HP por defecto.
    - contact_distance: distancia que define vecinos (por defecto 1.0).
    - tol: tolerancia numérica para comparar distancias.
    - metric: 'euclidean' (por defecto) o 'manhattan'.
    - return_contacts: si True, devuelve además lista de (i,j,contribución).

    Devuelve:
    - energía (float), o (energía, contactos) si return_contacts=True.
    """
    coords = np.asarray(coords, dtype=float)
    if coords.ndim != 2 or coords.shape[1] != 3:
        raise ValueError("coords debe ser array de forma (N,3)")
    N = coords.shape[0]
    if isinstance(seq, str):
        seq_list = list(seq.upper())
    else:
        seq_list = [str(s).upper() for s in seq]
    if len(seq_list) != N:
        raise ValueError("La longitud de seq debe coincidir con el número de coordenadas")

    # normalizar U a una función callable
    if U is None:
        Ucall = _default_U
    elif callable(U):
        Ucall = U
    elif isinstance(U, dict):
        def Ucall(a,b):
            a = a.upper(); b = b.upper()
            # intentar dict-of-dicts con simetría si es necesario
            if a in U and b in U[a]:
                return float(U[a][b])
            if b in U and a in U[b]:
                return float(U[b][a])
            # intentar clave tuple (a,b)
            if (a,b) in U:
                return float(U[(a,b)])
            if (b,a) in U:
                return float(U[(b,a)])
            # fallback
            return 0.0
    else:
        raise ValueError("U debe ser None, callable o dict-of-dicts")

    E = 0.0
    contacts: List[Tuple[int,int,float]] = []

    # Iterar sobre pares i<j con |i-j| >= 2 (i=0..N-3, j=i+2..N-1)
    for i, j in combinations(range(N), 2):
        if abs(i - j) < 2:
            continue  # saltar pares consecutivos
        diff = coords[i] - coords[j]
        if metric == 'euclidean':
            d = np.linalg.norm(diff)
            is_contact = abs(d - contact_distance) <= tol
        elif metric == 'manhattan':
            d = np.sum(np.abs(diff))
            is_contact = abs(d - contact_distance) <= tol
        else:
            raise ValueError("metric debe ser 'euclidean' o 'manhattan'")

        if is_contact:
            contrib = float(Ucall(seq_list[i], seq_list[j]))
            if contrib != 0.0:
                E += contrib
            # guardamos el contacto y la contribución (incluso si 0) para debug
            contacts.append((i, j, contrib))

    if return_contacts:
        return E, contacts
    return E

In [24]:
# Tests simples
coords1 = [(0,0,0),(1,0,0),(2,0,0),(3,0,0)]
seq1 = "HHHH"
print("Lineal (sin contactos):", energy_hp(coords1, seq1))  # 0

coords2 = [(0,0,0),(1,0,0),(1,1,0),(0,1,0)]
seq2 = "HHHH"
print("i=0,j=3 contacto:", energy_hp(coords2, seq2))  # -1

coords3 = [(0,0,0),(1,0,0),(2,0,0),(2,1,0),(1,1,0),(0,1,0)]
seq3 = "HHHHHH"
print("i=0,j=5 contacto y i=1,j=4 contacto:", energy_hp(coords3, seq3))  # -2

coords3 = [(0,0,0),(1,0,0),(2,0,0),(2,1,0),(1,1,0),(0,1,0)]
seq3 = "HHHHHP"
print("i=1,j=4 contacto:", energy_hp(coords3, seq3))  # -1

Lineal (sin contactos): 0.0
i=0,j=3 contacto: -1.0
i=0,j=5 contacto y i=1,j=4 contacto: -2.0
i=1,j=4 contacto: -1.0


In [25]:
# Para debugging: devolver índices de los contactos y contribución de cada uno
coords3 = [(0,0,0),(1,0,0),(2,0,0),(2,1,0),(1,1,0),(0,1,0)]
seq3 = "HHHHHH"
print("i=0,j=5 contacto y i=1,j=4 contacto:", energy_hp(coords3, seq3, return_contacts=True))  # -2

i=0,j=5 contacto y i=1,j=4 contacto: (-2.0, [(0, 5, -1.0), (1, 4, -1.0)])


## **Parte 3**: MonteCarlo Moves

In [21]:
import numpy as np
import random
from typing import List, Tuple, Optional

def check_self_avoiding(coords: List[Tuple[int, int, int]]) -> bool:
    """
    Check if the chain is self-avoiding (no duplicate positions).
    """
    return len(set(coords)) == len(coords)

def check_chain_connectivity(coords: List[Tuple[int, int, int]]) -> bool:
    """
    Check if the chain maintains connectivity (adjacent residues are lattice neighbors).
    """
    for i in range(len(coords) - 1):
        dx = abs(coords[i][0] - coords[i+1][0])
        dy = abs(coords[i][1] - coords[i+1][1])
        dz = abs(coords[i][2] - coords[i+1][2])
        
        # Adjacent residues should differ by exactly 1 in one coordinate
        if dx + dy + dz != 1:
            return False
    return True

def end_move(coords: List[Tuple[int, int, int]], hp_types: List[str]) -> Tuple[List[Tuple[int, int, int]], bool]:
    """
    Move an end residue to a new position.
    
    This is the simplest Monte Carlo move: take the first or last residue and move it
    to a new position adjacent to its bonded neighbor, while maintaining self-avoidance.
    
    Args:
        coords: Current coordinates of the chain
        hp_types: List of 'H' or 'P' for each residue
        
    Returns:
        new_coords: New coordinates after move (or unchanged if move fails)
        success: Boolean indicating if move was successful
    """
    n = len(coords)
    if n < 2:
        return coords, False
    
    # Choose which end to move (0 = N-terminus, n-1 = C-terminus)
    end_idx = random.choice([0, n-1])
    
    # Get the bonded neighbor
    neighbor_idx = 1 if end_idx == 0 else n-2
    current_pos = coords[end_idx]
    neighbor_pos = coords[neighbor_idx]
    
    # All possible lattice directions
    directions = [(1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1)]
    
    # Find all valid positions adjacent to the neighbor
    valid_positions = []
    for dx, dy, dz in directions:
        candidate = (neighbor_pos[0] + dx, neighbor_pos[1] + dy, neighbor_pos[2] + dz)
        
        # Check if position is unoccupied and different from current position
        if candidate != current_pos and candidate not in coords:
            valid_positions.append(candidate)
    
    if not valid_positions:
        return coords, False
    
    # Choose a random valid position
    new_pos = random.choice(valid_positions)
    
    # Create new coordinates
    new_coords = coords.copy()
    new_coords[end_idx] = new_pos
    
    # Verify the move maintains self-avoidance
    if not check_self_avoiding(new_coords):
        return coords, False
    
    return new_coords, True

def crankshaft_move(coords: List[Tuple[int, int, int]], hp_types: List[str]) -> Tuple[List[Tuple[int, int, int]], bool]:
    """
    Perform a crankshaft (corner flip) move.
    
    This move identifies a 3-residue L-shaped segment (i, i+1, i+2) and rotates
    the middle residue to the other possible position that maintains connectivity.
    
    For example, if we have:
    - residue i at (0,0,0)
    - residue i+1 at (1,0,0)  
    - residue i+2 at (1,1,0)
    
    The middle residue can be flipped to (0,1,0) to create:
    (0,0,0) -> (0,1,0) -> (1,1,0)
    
    Args:
        coords: Current coordinates of the chain
        hp_types: List of 'H' or 'P' for each residue
        
    Returns:
        new_coords: New coordinates after move (or unchanged if move fails)
        success: Boolean indicating if move was successful
    """
    n = len(coords)
    if n < 3:
        return coords, False
    
    # Find all valid 3-residue segments that form an L-shape (90-degree turn)
    valid_segments = []
    
    for i in range(n - 2):
        p0 = coords[i]
        p1 = coords[i+1]
        p2 = coords[i+2]
        
        # Check if this forms an L-shape (two perpendicular unit vectors)
        v1 = (p1[0] - p0[0], p1[1] - p0[1], p1[2] - p0[2])
        v2 = (p2[0] - p1[0], p2[1] - p1[1], p2[2] - p1[2])
        
        # Both vectors should be unit lattice vectors
        if sum(abs(x) for x in v1) != 1 or sum(abs(x) for x in v2) != 1:
            continue
            
        # Vectors should be perpendicular (dot product = 0)
        dot = v1[0]*v2[0] + v1[1]*v2[1] + v1[2]*v2[2]
        if dot == 0:
            valid_segments.append(i)
    
    if not valid_segments:
        return coords, False
    
    # Randomly choose a valid segment
    i = random.choice(valid_segments)
    
    p0 = coords[i]
    p1 = coords[i+1]
    p2 = coords[i+2]
    
    # Calculate the two vectors
    v1 = (p1[0] - p0[0], p1[1] - p0[1], p1[2] - p0[2])
    v2 = (p2[0] - p1[0], p2[1] - p1[1], p2[2] - p1[2])
    
    # The new position for p1 is: p0 + v2 (going from p0 in direction of v2)
    # OR equivalently: p2 + v1 (going from p2 in direction of v1)
    new_p1 = (p0[0] + v2[0], p0[1] + v2[1], p0[2] + v2[2])
    
    # Alternative: new_p1 = (p2[0] + v1[0], p2[1] + v1[1], p2[2] + v1[2])
    # Both should give the same result for a proper L-shape
    
    # Check if the new position is unoccupied
    # We need to check all positions except the current p1
    occupied = set(coords[:i+1] + coords[i+2:])
    
    if new_p1 in occupied:
        return coords, False
    
    # Create new coordinates
    new_coords = coords.copy()
    new_coords[i+1] = new_p1
    
    return new_coords, True

def pivot_move(coords: List[Tuple[int, int, int]], hp_types: List[str]) -> Tuple[List[Tuple[int, int, int]], bool]:
    """
    Pivot move: Choose a pivot point and rotate one part of the chain.
    
    This is a more global move that can dramatically change conformation.
    We randomly choose a pivot residue and rotate the tail (everything after pivot)
    by 90 or 180 degrees around an axis.
    
    Args:
        coords: Current coordinates of the chain
        hp_types: List of 'H' or 'P' for each residue
        
    Returns:
        new_coords: New coordinates after move (or unchanged if move fails)
        success: Boolean indicating if move was successful
    """
    n = len(coords)
    if n < 3:
        return coords, False
    
    # Choose a pivot point (not at the very end)
    pivot_idx = random.randint(1, n-2)
    
    # Choose rotation: 90 or 180 degrees around x, y, or z axis
    # Rotation matrices for 90 degree rotations in 3D lattice
    rotations = [
        # 90° around x-axis
        lambda x, y, z: (x, -z, y),
        # 90° around y-axis
        lambda x, y, z: (z, y, -x),
        # 90° around z-axis
        lambda x, y, z: (-y, x, z),
        # 180° around x-axis
        lambda x, y, z: (x, -y, -z),
        # 180° around y-axis
        lambda x, y, z: (-x, y, -z),
        # 180° around z-axis
        lambda x, y, z: (-x, -y, z)
    ]
    
    rot_func = random.choice(rotations)
    
    # Get pivot coordinates
    pivot = coords[pivot_idx]
    
    # Rotate the tail (residues after pivot)
    new_coords = coords.copy()
    for i in range(pivot_idx + 1, n):
        # Translate to origin, rotate, translate back
        dx = coords[i][0] - pivot[0]
        dy = coords[i][1] - pivot[1]
        dz = coords[i][2] - pivot[2]
        
        rx, ry, rz = rot_func(dx, dy, dz)
        new_pos = (pivot[0] + rx, pivot[1] + ry, pivot[2] + rz)
        
        # Check if this position is already occupied (except by residues we're moving)
        # We need to check all occupied positions except those in the tail we're rotating
        occupied_positions = set(coords[:pivot_idx + 1])  # Fixed part of chain
        
        if new_pos in occupied_positions:
            return coords, False  # Collision detected
        
        new_coords[i] = new_pos
    
    # Final check: ensure entire chain is self-avoiding
    if not check_self_avoiding(new_coords):
        return coords, False
    
    return new_coords, True

def mcmc_move(coords: List[Tuple[int, int, int]], hp_types: List[str], 
              move_type: str = 'random') -> Tuple[List[Tuple[int, int, int]], bool]:
    """
    Perform a Monte Carlo move of specified type.
    
    Args:
        coords: Current coordinates
        hp_types: List of 'H' or 'P' for each residue
        move_type: 'end', 'crankshaft', 'pivot', or 'random' (randomly choose)
        
    Returns:
        new_coords: New coordinates after move
        success: Boolean indicating if move was successful
    """
    if move_type == 'random':
        move_type = random.choice(['end', 'crankshaft', 'pivot'])
    
    if move_type == 'end':
        return end_move(coords, hp_types)
    elif move_type == 'crankshaft':
        return crankshaft_move(coords, hp_types)
    elif move_type == 'pivot':
        return pivot_move(coords, hp_types)
    else:
        raise ValueError(f"Unknown move type: {move_type}")

# Test the moves
if __name__ == "__main__":
    HP_MAP = {
        'A': 'H', 'C': 'H', 'I': 'H', 'L': 'H', 'M': 'H',
        'F': 'H', 'W': 'H', 'Y': 'H', 'V': 'H',
        'R': 'P', 'N': 'P', 'D': 'P', 'Q': 'P', 'E': 'P',
        'G': 'P', 'H': 'P', 'K': 'P', 'P': 'P', 'S': 'P', 'T': 'P'
    }
    
    # Test with a short sequence
    test_sequence = "ACDEFG"
    print(f"Test sequence: {test_sequence}")
    print(f"HP mapping: {[HP_MAP.get(aa, 'P') for aa in test_sequence]}")
    
    # Generate linear chain
    coords, hp_types = generate_linear_chain(test_sequence)
    print(f"\nInitial linear chain coordinates: {coords}")
    print(f"Self-avoiding: {check_self_avoiding(coords)}")
    print(f"Chain connectivity: {check_chain_connectivity(coords)}")
    
    # Test end move
    new_coords, success = end_move(coords, hp_types)
    print(f"\nEnd move success: {success}")
    if success:
        print(f"New coordinates: {new_coords}")
        print(f"Self-avoiding: {check_self_avoiding(new_coords)}")
        plot_hp_lattice_3d(test_sequence, new_coords, hp_types)
    
    # Test corner flip (will likely fail on linear chain - needs a corner)
    new_coords2, success2 = crankshaft_move(coords, hp_types)
    print(f"\nCorner flip success (on linear chain): {success2}")
    
    # Create a chain with a corner manually for testing
    coords_with_corner = [(0,0,0), (1,0,0), (1,1,0), (1,1,1)]
    hp_types_corner = ['H', 'P', 'H', 'P']
    new_coords3, success3 = crankshaft_move(coords_with_corner, hp_types_corner)
    print(f"\nCorner flip success (on chain with corner): {success3}")
    if success3:
        print(f"Original: {coords_with_corner}")
        plot_hp_lattice_3d("HPHP", coords_with_corner, hp_types_corner)
        print(f"New: {new_coords3}")
        plot_hp_lattice_3d("HPHP", new_coords3, hp_types_corner)
    
    # Test pivot move
    new_coords4, success4 = pivot_move(coords, hp_types)
    print(f"\nPivot move success: {success4}")
    if success4:
        print(f"Original: {coords}")
        plot_hp_lattice_3d(test_sequence, coords, hp_types)
        print(f"New: {new_coords4}")
        plot_hp_lattice_3d(test_sequence, new_coords4, hp_types)

Test sequence: ACDEFG
HP mapping: ['H', 'H', 'P', 'P', 'H', 'P']

Initial linear chain coordinates: [(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0)]
Self-avoiding: True
Chain connectivity: True

End move success: True
New coordinates: [(1, 0, 1), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0)]
Self-avoiding: True


Sequence: ACDEFG
Length: 6 residues
H residues: 3
P residues: 3
Unique positions: 6

Corner flip success (on linear chain): False

Corner flip success (on chain with corner): True
Original: [(0, 0, 0), (1, 0, 0), (1, 1, 0), (1, 1, 1)]


Sequence: HPHP
Length: 4 residues
H residues: 2
P residues: 2
Unique positions: 4
New: [(0, 0, 0), (1, 0, 0), (1, 0, 1), (1, 1, 1)]


Sequence: HPHP
Length: 4 residues
H residues: 2
P residues: 2
Unique positions: 4

Pivot move success: True
Original: [(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (5, 0, 0)]


Sequence: ACDEFG
Length: 6 residues
H residues: 3
P residues: 3
Unique positions: 6
New: [(0, 0, 0), (1, 0, 0), (2, 0, 0), (3, 0, 0), (4, 0, 0), (4, 0, -1)]


Sequence: ACDEFG
Length: 6 residues
H residues: 3
P residues: 3
Unique positions: 6


## **Parte 5**: Comparativo con proteínas reales

In [ ]:
# Real protein examples for your project
REAL_PROTEINS = {
    "Trp-cage": {
        "seq": "NLYIQWLKDGGPSSGRPPPS",
        "pdb": "1L2Y",
        "length": 20,
        "notes": "Fast-folding, hydrophobic core with Trp"
    },
    "Beta-hairpin": {
        "seq": "RGKGRRLLGKGRR",
        "pdb": "1E0G",  # Actually part of protein G
        "length": 13,
        "notes": "Forms antiparallel beta-sheet"
    },
    "Alpha-helix": {
        "seq": "MKALEEKELKALEEKLKALEE",
        "pdb": "Custom",
        "length": 21,
        "notes": "Designed alpha-helical peptide"
    }
}